In [12]:
import FinanceDataReader as fdr

In [15]:
# 삼성전자 종목코드 : 005930
df1 = fdr.DataReader("005930")
df1.shape

(6000, 6)

In [16]:
# SK하이닉스 종목코드 : 000660
df2 = fdr.DataReader("000660", start = '2018-04-05', end = '2021-05-31')
df2.shape

(775, 6)

## 한국 거래소 상장 종목 정보 불러오기

In [17]:
stock_info = fdr.StockListing("KRX")
stock_info.shape

(7561, 10)

In [20]:
stock_info.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


In [38]:
def stock_listing():
    import requests
    import json
    try:
        from pandas import json_normalize
    except ImportError:
        from pandas.io.json import json_normalize

    data = {'bld': 'dbms/comm/finder/finder_stkisu', 'locale': 'ko_KR', }

    r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', data=data)
    jo = json.loads(r.text)
    df = json_normalize(jo, 'block1')
    df_info = df[['short_code', 'codeName']]
    df_info.columns = ['Symbol', 'Name']
    return df_info

In [41]:
stock_info = stock_listing()
stock_info

,Symbol,Name
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
7556,435540,히어로즈 TDF2040액티브
7557,435550,히어로즈 TDF2050액티브
7558,419890,히어로즈 단기채권ESG액티브
7559,429870,히어로즈 리츠이지스액티브


# 종목에 해당하는 종목코드 찾기

In [42]:
# cf. '바나나'에 해당되는 정보찾기
import pandas as pd

fruits = ['사과', '바나나', '딸기', '복숭아', '수박', '참외', '포도']
price = [1000, 2000, 3000, 4000, 5000, 6000, 7000]
market = ['A마트', 'B마트', 'C마트', 'D마트', 'E마트', 'F마트', 'G마트']
df = pd.DataFrame({"과일":fruits, "가격":price, "파는곳":market})
df.index = range(1,8)

# condition = df['과일'] == '바나나'
# df[condition]['가격'].values[0]

In [43]:
stock_name = "AJ네트웍스"
stock_info[stock_info['Name'] == stock_name]['Symbol'].values[0]

'095570'

In [44]:
stock_name = "SK하이닉스"
if stock_name in stock_info['Name'].values:
    stock_symbol = stock_info[stock_info['Name']== stock_name]['Symbol'].values[0]
else:
    print("There is no name in KRX")
df = fdr.DataReader(stock_symbol, start = '2021')
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,124500,128000,120500,126000,7995016,0.063291
2021-01-05,124500,132500,124000,130500,7180224,0.035714
2021-01-06,132500,137000,130000,131000,8525749,0.003831
2021-01-07,132500,137500,131500,134500,5981299,0.026718
2021-01-08,136000,138000,132500,138000,8713010,0.026022


# StockListing()를 저장하고 불러올 수 있는 코드를 작성

In [45]:
# cf. 날짜를 다뤄보자.
import datetime
today_year = datetime.datetime.today().year
today_month = datetime.datetime.today().month
today_day = datetime.datetime.today().day
print(today_year, today_month, today_day)

2022 7 17


In [46]:
krx_df = stock_listing()
krx_df

,Symbol,Name
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
7556,435540,히어로즈 TDF2040액티브
7557,435550,히어로즈 TDF2050액티브
7558,419890,히어로즈 단기채권ESG액티브
7559,429870,히어로즈 리츠이지스액티브


In [47]:
krx_df.to_csv(f"krx_df_{today_year}_{today_month}_{today_day}.csv")

In [48]:
import os
os.path.isfile(f"krx_df_{today_year}_{today_month}_{today_day}.csv")

True

In [51]:
if os.path.isfile(f"krx_df_{today_year}_{today_month}_{today_day}.csv"):
    krx_df = pd.read_csv(f"krx_df_{today_year}_{today_month}_{today_day}.csv", index_col = 0)
else:
    krx_df = stock_listing()
    krx_df.to_csv(f"krx_df_{today_year}_{today_month}_{today_day}.csv")

In [52]:
krx_df

,Symbol,Name
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템
...,...,...
7556,435540,히어로즈 TDF2040액티브
7557,435550,히어로즈 TDF2050액티브
7558,419890,히어로즈 단기채권ESG액티브
7559,429870,히어로즈 리츠이지스액티브


# 함수 만들기_(stockDataReader_fn)

In [53]:
def stock_fdr_fn(stock_name, start_date = None, end_date =None):
    import FinanceDataReader as fdr
    import datetime
    import os
    import pandas as pd
    
    def stock_listing():
    
        import requests
        import json

        try:
            from pandas import json_normalize
        except ImportError:
            from pandas.io.json import json_normalize

        data = {'bld': 'dbms/comm/finder/finder_stkisu', 'locale': 'ko_KR', }

        r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', data=data)
        jo = json.loads(r.text)
        df = json_normalize(jo, 'block1')
        df_info = df[['short_code', 'codeName']]

        df_info.columns = ['Symbol', 'Name']

        return df_info
    
    #1. 저장 파일 형식에 필요한 Data 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day
    
    # 2. KRX 종목 DataFrame 존재 여부 확인
    if os.path.isfile(f"krx_df_{today_year}_{today_month}_{today_day}.csv"):
        krx_df = pd.read_csv(f"krx_df_{today_year}_{today_month}_{today_day}.csv", index_col = 0)
    else:
        krx_df = stock_listing()
        krx_df.to_csv(f"krx_df_{today_year}_{today_month}_{today_day}.csv")
    
    # 3. 종목 코드 찾기
    if stock_name in krx_df['Name'].values:
        symbol = krx_df[krx_df['Name'] == stock_name]['Symbol'].values[0]
    else:
        raise "Threr is no name in KRX"
    
    stock_df = fdr.DataReader(symbol, start_date, end_date)
    
    return stock_df

In [55]:
stock_name = "LG전자"
start_date = '2020'
end_date = None
df = stock_fdr_fn(stock_name = stock_name, start_date = start_date, end_date = end_date)
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,71800,72100,71000,71000,284758,-0.015257
2020-01-03,71400,71900,70800,71100,369493,0.001408
2020-01-06,70300,70500,69600,69900,337026,-0.016878
2020-01-07,69800,70600,69800,70300,265311,0.005722
2020-01-08,70300,70300,67000,67500,937438,-0.039829
...,...,...,...,...,...,...
2022-07-11,91500,92700,91000,91300,710884,-0.007609
2022-07-12,90700,90900,89100,90400,654534,-0.009858
2022-07-13,89900,92300,89900,91100,676772,0.007743
